In [73]:
import pandas as pd

import requests
from pprint import pprint
from statistics import mean
from datetime import datetime
import numpy as np
from scipy import stats

In [124]:
def build_url(lat, lon, start_date, end_date):
    url = 'https://archive-api.open-meteo.com/v1/era5?latitude={}&longitude={}&start_date={}&end_date={}'\
    '&hourly=temperature_2m&hourly=surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,relativehumidity_2m,precipitation'\
        '&daily=weathercode,'\
            'temperature_2m_max,'\
            'temperature_2m_min,'\
            'apparent_temperature_max,'\
            'apparent_temperature_min,'\
            'sunrise,'\
            'sunset,'\
            'shortwave_radiation_sum,'\
            'precipitation_sum,'\
            'rain_sum,'\
            'snowfall_sum,'\
            'precipitation_hours,'\
            'windspeed_10m_max,'\
            'windgusts_10m_max,'\
            'winddirection_10m_dominant,'\
            'et0_fao_evapotranspiration'\
        '&timezone=America%2FSao_Paulo'
    return url.format(lat, lon, start_date, end_date)

def get_meteo_data(lat, lon, start_date, end_date):
    url = build_url(lat, lon, start_date, end_date)
    response = requests.get(url)
    meteo_json = response.json()
    return meteo_json

In [95]:
df = pd.read_csv('data/orders.csv')

df.head()

,timestamp,payment_value,zip_code,city,state,latitude,longitude,items_quant,product_category,date,time,weekday
0,2017-10-02 10:56:33,18.12,3149.0,sao paulo,SP,-23.576983,-46.587161,1.0,housewares,2017-10-02,10:56:33,Monday
1,2017-10-02 10:56:33,2.00,3149.0,sao paulo,SP,-23.576983,-46.587161,1.0,housewares,2017-10-02,10:56:33,Monday
2,2017-10-02 10:56:33,18.59,3149.0,sao paulo,SP,-23.576983,-46.587161,1.0,housewares,2017-10-02,10:56:33,Monday
3,2018-07-24 20:41:37,141.46,47813.0,barreiras,BA,-12.177924,-44.660711,1.0,perfumery,2018-07-24,20:41:37,Tuesday
4,2018-08-08 08:38:49,179.12,75265.0,vianopolis,GO,-16.745150,-48.514783,1.0,auto,2018-08-08,08:38:49,Wednesday


In [96]:
df['apparent_temperature_max'] = np.nan
df['apparent_temperature_min'] = np.nan
df['et0_fao_evapotranspiration'] = np.nan
df['precipitation_hours'] = np.nan
df['precipitation_sum'] = np.nan
df['rain_sum'] = np.nan
df['snowfall_sum'] = np.nan
df['sunrise'] = np.nan
df['sunset'] = np.nan
df['weathercode'] = np.nan
df['winddirection_10m_dominant'] = np.nan
df['windgusts_10m_max'] = np.nan
df['windspeed_10m_max'] = np.nan
df['cloud_cover_total_mean_%'] = np.nan
df['cloudcover_high_mean_%'] = np.nan
df['cloudcover_low_mean_%'] = np.nan
df['cloudcover_mid_mean_%'] = np.nan
df['surface_pressure_mean_hPa'] = np.nan
df['temperature_2m_max'] = np.nan
df['temperature_2m_min'] = np.nan
df['daily_shortwave_radiation_sum'] = np.nan
df['temperature_2m_hourly'] = np.nan
df['precipitation_hourly'] = np.nan
df['relative_humidity_2m_hourly'] = np.nan

In [97]:
for idx in df.index:
    date = pd.to_datetime(df['timestamp'][idx]).date().strftime('%Y-%m-%d')
    meteo_data = get_meteo_data(
        df['latitude'][idx],
        df['longitude'][idx],
        date,
        date
    )
    df['apparent_temperature_max'][idx] = meteo_data['daily']['apparent_temperature_max'][0]
    df['apparent_temperature_min'][idx] = meteo_data['daily']['apparent_temperature_min'][0]
    df['et0_fao_evapotranspiration'][idx] = meteo_data['daily']['et0_fao_evapotranspiration'][0]
    df['precipitation_hours'][idx] = meteo_data['daily']['precipitation_hours'][0]
    df['precipitation_sum'][idx] = meteo_data['daily']['precipitation_sum'][0]
    df['rain_sum'][idx] = meteo_data['daily']['rain_sum'][0]
    df['snowfall_sum'][idx] = meteo_data['daily']['snowfall_sum'][0]
    df['sunrise'][idx] = meteo_data['daily']['sunrise'][0]
    df['sunset'][idx] = meteo_data['daily']['sunset'][0]
    df['weathercode'][idx] = meteo_data['daily']['weathercode'][0]
    df['winddirection_10m_dominant'][idx] = meteo_data['daily']['winddirection_10m_dominant'][0]
    df['windgusts_10m_max'][idx] = meteo_data['daily']['windgusts_10m_max'][0]
    df['windspeed_10m_max'][idx] = meteo_data['daily']['windspeed_10m_max'][0]
    df['cloud_cover_total_mean_%'][idx] = mean(meteo_data['hourly']['cloudcover'][8:20])
    df['cloudcover_high_mean_%'][idx] = mean(meteo_data['hourly']['cloudcover_high'][8:20])
    df['cloudcover_low_mean_%'][idx] = mean(meteo_data['hourly']['cloudcover_low'][8:20])
    df['cloudcover_mid_mean_%'][idx] = mean(meteo_data['hourly']['cloudcover_mid'][8:20])
    df['surface_pressure_mean_hPa'][idx] = mean(meteo_data['hourly']['surface_pressure'][8:20])
    
    hour_val = pd.to_datetime(df['timestamp'][idx]).time().hour
    df['temperature_2m_max'][idx] = meteo_data['daily']['temperature_2m_max'][0]
    df['temperature_2m_min'][idx] = meteo_data['daily']['temperature_2m_min'][0]
    df['daily_shortwave_radiation_sum'][idx] = meteo_data['daily']['shortwave_radiation_sum'][0]
    df['temperature_2m_hourly'][idx] = meteo_data['hourly']['temperature_2m'][hour_val]
    df['precipitation_hourly'][idx] = meteo_data['hourly']['precipitation'][hour_val]
    df['relative_humidity_2m_hourly'][idx] = meteo_data['hourly']['relativehumidity_2m'][hour_val]
        
    finish_percent = idx % 100
    if (finish_percent == 0):
        print(idx / len(df) * 100)

    


/var/folders/f1/914jm09n78n0_j66rqq26xw40000gn/T/ipykernel_45009/3977165480.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['apparent_temperature_max'][idx] = meteo_data['daily']['apparent_temperature_max'][0]
/var/folders/f1/914jm09n78n0_j66rqq26xw40000gn/T/ipykernel_45009/3977165480.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['apparent_temperature_min'][idx] = meteo_data['daily']['apparent_temperature_min'][0]
/var/folders/f1/914jm09n78n0_j66rqq26xw40000gn/T/ipykernel_45009/3977165480.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

0.0
0.09276265746461106
0.1855253149292221
0.2782879723938331
0.3710506298584442
0.46381328732305527
0.5565759447876663
0.6493386022522774
0.7421012597168885
0.8348639171814994
0.9276265746461105
1.0203892321107215
1.1131518895753325
1.2059145470399437
1.2986772045045547
1.3914398619691657
1.484202519433777
1.5769651768983879
1.6697278343629989
1.7624904918276099
1.855253149292221
1.948015806756832
2.040778464221443
2.1335411216860543
2.226303779150665
2.3190664366152762
2.4118290940798874
2.504591751544498
2.5973544090091094
2.6901170664737206
2.7828797239383314
2.875642381402942
2.968405038867554
3.0611676963321646
3.1539303537967758
3.246693011261387
3.3394556687259978
3.4322183261906085
3.5249809836552197
3.6177436411198305
3.710506298584442
3.803268956049053
3.896031613513664
3.988794270978275
4.081556928442886
4.174319585907496
4.2670822433721085
4.35984490083672
4.45260755830133
4.545370215765941
4.6381328732305525
4.730895530695163
4.823658188159775
4.916420845624385
5.00918350

41.27938257175192
41.37214522921653
41.46490788668113
41.55767054414575
41.65043320161036
41.74319585907497
41.835958516539584
41.92872117400419
42.02148383146881
42.11424648893342
42.20700914639803
42.29977180386263
42.392534461327244
42.48529711879186
42.578059776256474
42.670822433721085
42.76358509118569
42.8563477486503
42.94911040611492
43.04187306357953
43.134635721044134
43.227398378508745
43.320161035973356
43.412923693437975
43.50568635090258
43.59844900836719
43.6912116658318
43.78397432329641
43.87673698076103
43.969499638225635
44.062262295690246
44.15502495315486
44.24778761061947
44.34055026808408
44.43331292554869
44.5260755830133
44.61883824047791
44.711600897942525
44.804363555407136
44.89712621287175
44.98988887033636
45.08265152780097
45.17541418526558
45.26817684273019
45.3609395001948
45.453702157659414
45.54646481512402
45.63922747258864
45.73199013005325
45.82475278751786
45.91751544498246
46.010278102447074
46.10304075991169
46.195803417376304
46.28856607484091

83.48639171814995
83.57915437561455
83.67191703307917
83.76467969054377
83.85744234800838
83.950205005473
84.04296766293761
84.13573032040222
84.22849297786684
84.32125563533144
84.41401829279606
84.50678095026066
84.59954360772527
84.69230626518988
84.78506892265449
84.8778315801191
84.97059423758373
85.06335689504833
85.15611955251295
85.24888220997755
85.34164486744217
85.43440752490677
85.52717018237138
85.619932839836
85.7126954973006
85.80545815476522
85.89822081222984
85.99098346969444
86.08374612715906
86.17650878462366
86.26927144208827
86.36203409955289
86.45479675701749
86.54755941448211
86.64032207194671
86.73308472941133
86.82584738687595
86.91861004434055
87.01137270180516
87.10413535926978
87.19689801673438
87.289660674199
87.3824233316636
87.4751859891282
87.56794864659282
87.66071130405744
87.75347396152206
87.84623661898667
87.93899927645127
88.03176193391589
88.12452459138049
88.2172872488451
88.31004990630971
88.40281256377432
88.49557522123894
88.58833787870356
88.

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107802 entries, 0 to 107801
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   timestamp                      107802 non-null  object 
 1   payment_value                  107802 non-null  float64
 2   zip_code                       107802 non-null  float64
 3   city                           107802 non-null  object 
 4   state                          107802 non-null  object 
 5   latitude                       107802 non-null  float64
 6   longitude                      107802 non-null  float64
 7   items_quant                    107802 non-null  float64
 8   product_category               107802 non-null  object 
 9   date                           107802 non-null  object 
 10  time                           107802 non-null  object 
 11  weekday                        107802 non-null  object 
 12  apparent_temperature_max      

In [154]:
df_copy.to_csv('data/orders_with_meteo.csv')